In [0]:
# Exemplo: instalação de uma biblioteca no cluster
# %pip install feature-engine lightgbm (instala biblioteca no cluster do bricks)
# %pip install feature-engine borutapy (instala biblioteca no cluster do bricks)

from feature_selection import *
from utils import *

# Base de Exemplo

In [0]:
from sklearn.datasets import make_classification
import numpy as np
import pandas as pd

# Cria variáveis numéricas usadas durante o estudo
X, y = make_classification(
    n_samples = 10000,
    n_features = 80,
    n_informative = 20,
    n_redundant = 60,
    n_clusters_per_class = 1,
    shuffle = False,
    random_state = 42
)

# Converte para DataFrame
df = pd.DataFrame(X, columns = [f'feat_{i}' for i in range(1, 81)])

# Cria 10 colunas com variáveis numéricas com apenas 10% de preenchimento
for i in range(10):
    df[f'feat_{81+i}'] = np.where(
        np.random.rand(len(df)) < 0.05,
        np.random.rand(len(df)),
        np.nan
    )

# Cria 10 colunas com variáveis numéricas com 95% dos valores definidos como 1 e o restante como 0
for i in range(10):
    df[f'feat_{91+i}'] = np.where(
        np.random.rand(len(df)) < 0.95,
        1, 
        0
    )

# df['feat_101'] = 'Teste variáveis string'

df['target'] = y

display(df.head(3))

In [0]:
df_na, relatorio_na = remove_features_na(
    df,
    threshold = 0.9, 
    target_col = 'target'
)

df_sem_variacao_percentil, relatorio_sem_variacao_percentil = remove_features_sem_variacao_percentil(
    df_na, 
    p = 0.1, 
    target_col = 'target'
)

df_selecao_feature_gain, relatorio_feature_gain = seleciona_top_features_information_gain(
    df_sem_variacao_percentil, 
    target_col = 'target', 
    importance_threshold = 0.8
)

df_selecao_pearson, relatorio_pearson = remover_variaveis_correlacionadas(
    df_selecao_feature_gain,
    target_col = 'target',
    corr_threshold = 0.5,
    method = 'pearson'
)

df_selecao_spearman, relatorio_spearman = remover_variaveis_correlacionadas(
    df_selecao_pearson,
    target_col = 'target',
    corr_threshold = 0.5,
    method = 'spearman'
)

df_selecao_boruta, relatorio_boruta = selecionar_features_boruta(
    df_selecao_spearman,
    target_col = 'target'
)

relatorio_selecionadas = relatorio_selecionadas(
    df_selecao_boruta, 
    'target'
)

In [0]:
display(
    pd.concat(
        [
            relatorio_na,
            relatorio_sem_variacao_percentil,
            relatorio_feature_gain,
            relatorio_pearson,
            relatorio_spearman,
            relatorio_boruta,
            relatorio_selecionadas
        ],
        ignore_index=True
    )
)